In [1]:
import gc
import os
import logging
import csv
import pandas as pd
import numpy as np
from pandas.core.interchange.dataframe_protocol import DataFrame

In [2]:
def step(in_path, out_path, func):
    print(f"\n➡️  Running: {func.__name__} ...")
    
    df = pd.read_parquet(in_path)
    df = func(df)
    df.to_parquet(out_path, index=False)
    
    del df
    gc.collect()
    print(f"✅ Saved to: {out_path}")

In [3]:
def initial_checks(df: pd.DataFrame):
    print(df.shape)
    # remove leading spaces in column definitions
    df.columns = [col[1:] if col.startswith(' ') else col for col in df.columns]
    
    # check for mixed data types in all columns
    columns = df.columns
    for c in columns:
        unique_types = set(df[c].map(type))
        assert len(unique_types) == 1, f"Error: multiple data types detected in {c} "
    del columns
    print("No mixed data in columns detected")

    # number of occurances for each Label
    labels = df['Label'].unique()
    data = []
    total_count = df.shape[0]
    for label in labels:
        count = df[df['Label'] == label].shape[0]
        data.append([label, count, round(count/total_count, 6)])
    label_data = pd.DataFrame(data=np.array(data), columns=['Label', 'Number of instances', '% of total instances'])
    print(label_data)
    label_data.to_csv('../data/statistics/label_data_original.csv')
    return df

In [4]:
#df_csv = pd.read_csv('../data/CICIDS2017_original.csv')
df_csv_checks = initial_checks(pd.read_csv('../data/originals/CICIDS2017.csv'))
# del df_csv
df_csv_checks.to_parquet('../data/originals/CICIDS2017.parquet', index=False)
del df_csv_checks
gc.collect()

(2830743, 79)
No mixed data in columns detected
                         Label Number of instances % of total instances
0                       BENIGN             2273097             0.803004
1                         DDoS              128027             0.045227
2                     PortScan              158930             0.056144
3                          Bot                1966             0.000695
4                 Infiltration                  36              1.3e-05
5     Web Attack � Brute Force                1507             0.000532
6             Web Attack � XSS                 652              0.00023
7   Web Attack � Sql Injection                  21                7e-06
8                  FTP-Patator                7938             0.002804
9                  SSH-Patator                5897             0.002083
10               DoS slowloris                5796             0.002048
11            DoS Slowhttptest                5499             0.001943
12              

34

In [5]:
df = pd.read_parquet('../data/originals/CICIDS2017.parquet')

In [6]:
def drop_nan_columns(df: pd.DataFrame):
    # Drop Columns with only NaN-Values
    nan_columns = df.columns[df.isna().all()].to_list()
    df = df.drop(columns = nan_columns)
    print(f'Deleted  {len(nan_columns)} NaN Columns: {nan_columns}')
    return df

In [7]:
step('../data/originals/CICIDS2017.parquet','../data/tmp/step_01.parquet', drop_nan_columns)


➡️  Running: drop_nan_columns ...
Deleted  0 NaN Columns: []
✅ Saved to: ../data/tmp/step_01.parquet


In [8]:
def drop_single_value_columns(df: pd.DataFrame):
    # Drop Columns containing the same value in each instance
    single_value_columns = df.columns[df.nunique(dropna=False) <= 1].to_list()
    df = df.drop(columns = single_value_columns)
    print(f'Deleted  {len(single_value_columns)} single-value Columns: {single_value_columns}')
    return df

In [9]:
step('../data/tmp/step_01.parquet','../data/tmp/step_02.parquet', drop_single_value_columns)


➡️  Running: drop_single_value_columns ...
Deleted  8 single-value Columns: ['Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']
✅ Saved to: ../data/tmp/step_02.parquet


In [10]:
def drop_duplicates(df: pd.DataFrame):
    # Drop duplicate instances
    count = df.duplicated(keep='first').sum()
    df = df.drop_duplicates(keep='first')
    print(f'Deleted  {count} duplicate instances')
    return df

In [11]:
step('../data/tmp/step_02.parquet','../data/tmp/step_03.parquet', drop_duplicates)


➡️  Running: drop_duplicates ...
Deleted  308381 duplicate instances
✅ Saved to: ../data/tmp/step_03.parquet


In [12]:
def drop_infinity_instances(df: pd.DataFrame):
    # Drop instances containing infinity values
    mask = df.isin([np.inf, -np.inf]).any(axis=1)
    count = mask.sum()
    df = df[~mask]
    print(f'Deleted  {count} instances with infinite values')
    return df

In [13]:
step('../data/tmp/step_03.parquet','../data/tmp/step_04.parquet', drop_infinity_instances)


➡️  Running: drop_infinity_instances ...
Deleted  1564 instances with infinite values
✅ Saved to: ../data/tmp/step_04.parquet


In [14]:
def drop_nan_instances(df: pd.DataFrame):
    # Drop instances containing NaN values
    count = df.isna().any(axis=1).sum()
    df = df.dropna()
    print(f'Deleted  {count} instances with NaN values')
    return df

In [15]:
step('../data/tmp/step_04.parquet','../data/tmp/step_05.parquet', drop_nan_instances)


➡️  Running: drop_nan_instances ...
Deleted  0 instances with NaN values
✅ Saved to: ../data/tmp/step_05.parquet


In [16]:
def merge_labels(df: pd.DataFrame):
    # reduce the number of unique labels by merging
    new_labels = {
        'BENIGN': 'BENIGN',
        'DDoS': 'Dos/DDos',
        'PortScan':'PortScan',
        'Bot': 'Bot',
        'Infiltration': 'Infiltration',
        'Web Attack � Brute Force': 'Web Attack',
        'Web Attack � XSS': 'Web Attack',
        'Web Attack � Sql Injection': 'Web Attack',
        'FTP-Patator': 'Brute Force',
        'SSH-Patator': 'Brute Force',
        'DoS slowloris': 'Dos/DDos',
        'DoS Slowhttptest': 'Dos/DDos',
        'DoS Hulk': 'Dos/DDos',
        'DoS GoldenEye': 'Dos/DDos',
        'Heartbleed':  'Heartbleed',
    }
    df['Label'] = df['Label'].map(new_labels)
    print(df['Label'].unique())
    return df

In [17]:
step('../data/tmp/step_05.parquet','../data/tmp/step_06.parquet', merge_labels)


➡️  Running: merge_labels ...
['BENIGN' 'Dos/DDos' 'PortScan' 'Bot' 'Infiltration' 'Web Attack'
 'Brute Force' 'Heartbleed']
✅ Saved to: ../data/tmp/step_06.parquet


In [18]:
# remove unnecessary Feature 'Destination Port' and save cleaned DataFrame
df = pd.read_parquet('../data/tmp/step_06.parquet')
df = df.drop(columns='Destination Port')
df.to_parquet('../data/CICIDS2017_cleaned.parquet', index=False)

In [19]:
# number of occurances for each Label for the cleaned DataFrame
df = pd.read_parquet('../data/CICIDS2017_cleaned.parquet')
labels = df['Label'].unique()
print(labels)
data = []
total_count = df.shape[0]
for label in labels:
    count = df[df['Label'] == label].shape[0]
    data.append([label, count, round(count/total_count, 6)])
label_data = pd.DataFrame(data=np.array(data), columns=['Label', 'Number of instances', '% of total instances'])
print(label_data)
label_data.to_csv('../data/statistics/label_data_cleaned.csv')

['BENIGN' 'Dos/DDos' 'PortScan' 'Bot' 'Infiltration' 'Web Attack'
 'Brute Force' 'Heartbleed']
          Label Number of instances % of total instances
0        BENIGN             2095057             0.831109
1      Dos/DDos              321759             0.127642
2      PortScan               90694             0.035978
3           Bot                1948             0.000773
4  Infiltration                  36              1.4e-05
5    Web Attack                2143              0.00085
6   Brute Force                9150              0.00363
7    Heartbleed                  11                4e-06
